<a href="https://colab.research.google.com/github/shuseiyokoi/data-science-portfolio/blob/main/cppi-simulation/cppi_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
np.random.seed(44)

In [2]:
initial_wealth = 100000
floor = 80000

In [3]:

def test (multiplier = 3, num_periods = 252, annual_return = 0.08, annual_volatility = 0.20, risk_free_rate = 0.02, balance_in = 30):
        for i in range(100) :
            # Frequency to balance portfolio in days np.random.seed(44)
            daily_risk_return = np.random.normal(annual_return / num_periods, annual_volatility / np.sqrt(num_periods), num_periods)

            # Create vectors of portfolio, safe asset, and risky asset values and give initial points
            portfolio_value = np.zeros(num_periods)
            portfolio_value[0] = initial_wealth

            safe_asset_value = np.zeros(num_periods)
            safe_asset_value[0] = initial_wealth - (initial_wealth - floor) * multiplier

            risky_asset_value = np.zeros(num_periods)
            risky_asset_value[0] = (initial_wealth - floor) * multiplier


            for t in range(1, num_periods):

                # Allocate to risky asset based on the cushion and multiplier
                risky_allocation = risky_asset_value[t-1]
                safe_allocation = safe_asset_value[t-1]

                # Calculate returns
                risky_asset_return = risky_allocation * (1 + daily_risk_return[t])
                safe_asset_return = safe_allocation * (1 + risk_free_rate / num_periods)

                # Update portfolio value
                portfolio_value[t] = risky_asset_return + safe_asset_return
                risky_asset_value[t] = risky_asset_return
                safe_asset_value[t] = safe_asset_return

                # Run below code by given frequency days
                if t % balance_in == 0:

                    # Calculate current cushion
                    cushion = max(portfolio_value[t-1] - floor, 0)

                    # Allocate to risky asset based on the cushion and multiplier
                    risky_allocation = multiplier * cushion
                    safe_allocation = portfolio_value[t-1] - risky_allocation

                    # Calculate returns
                    risky_asset_return = risky_allocation * (1 + daily_risk_return[t])
                    safe_asset_return = safe_allocation * (1 + risk_free_rate / num_periods)

                    # Update portfolio value
                    portfolio_value[t] = risky_asset_return + safe_asset_return
                    risky_asset_value[t] = risky_asset_return
                    safe_asset_value[t] = safe_asset_return

                    # Adjust allocations if the floor is breached
                    if portfolio_value[t] < floor:
                        portfolio_value[t] = floor
                        risky_asset_value[t] = 0
                        safe_asset_value[t] = floor


        # Plot the results
        plt.figure(figsize=(10, 6))
        plt.plot(portfolio_value, label='Total Portfolio Value')
        plt.plot(risky_asset_value, label='Risky Asset Value')
        plt.plot(safe_asset_value, label='Safe Asset Value')
        plt.axhline(y=floor, color='r', linestyle='--', label='Floor')
        plt.title('CPPI Strategy Simulation (Balance out in every ' + str(balance_in) +' days)')
        plt.xlabel('Days')
        plt.ylabel('Portfolio Value')
        plt.legend()
        plt.show()

test_controls = widgets.interactive(test,
                                    multiplier = (1, 5, .5),
                                    num_periods = 252,
                                    annual_return = (0., +.2, .01),
                                    annual_volatility = (0, 1, .05),
                                    risk_free_rate = (0, .05, .01),
                                    balance_in = (0, 252, 1)
)
display(test_controls)

interactive(children=(FloatSlider(value=3.0, description='multiplier', max=5.0, min=1.0, step=0.5), IntSlider(…